In [1]:
import ete3
import re
import itertools
import multiprocessing
import random

import pandas as pd
import numpy  as np
import igraph as ig
import pickle as pkl

from scipy.spatial.distance import squareform, pdist
from scipy.stats            import mannwhitneyu
from collections            import Counter

ncbi = ete3.NCBITaxa()
%cd ~/work2/PVCtree/

/nobackup1c/users/payette/PVCtree


In [2]:
#sampled_genomes = pd.read_csv('genomes.tab',
#                              sep='\t',
#                              index_col=0)

In [3]:
PVCtree_sampled_genomes = pd.read_csv('PVC_group_Rprots.csv', index_col=1)

In [4]:
PVCtree_sampled_genomes

,Index,URL,Filename,FullURL,Accession,TaxID
Name,,,,,,
Candidatus Brocadia sp.,0,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,GCA_016699285.1_ASM1669928v1_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,GCA_016699285.1 ASM1669928v1,2172549
Candidatus Brocadiae bacterium,1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,GCA_013359215.1_ASM1335921v1_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,GCA_013359215.1 ASM1335921v1,2517210
Phycisphaera mikurensis NBRC 102666,2,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_000284115.1_ASM28411v1_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCA_000284115.1 ASM28411v1,547188
Sedimentisphaerales bacterium,3,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,GCA_016932895.1_ASM1693289v1_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,GCA_016932895.1 ASM1693289v1,2813034
Humisphaera borealis,4,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_015169395.1_ASM1516939v1_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCA_015169395.1 ASM1516939v1,2807512
...,...,...,...,...,...,...
Oligosphaeraceae bacterium,101,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,GCA_012799725.1_ASM1279972v1_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,GCA_012799725.1 ASM1279972v1,2699751
Victivallales bacterium,102,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,GCA_018648945.1_ASM1864894v1_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,GCA_018648945.1 ASM1864894v1,2212475
Lentisphaerae bacterium GWF2_38_69,103,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,GCA_001803205.1_ASM180320v1_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,GCA_001803205.1 ASM180320v1,1798570


In [5]:
PVCtree_sampled_genomes['TaxID'].unique()

array([2172549, 2517210,  547188, 2813034, 2807512,     122,     114,
           128,  265606,     119,     120,  360056, 2528007, 2527989,
       2528027, 2527983, 2527996, 1891926, 2527994, 2527995, 1331910,
       1387353,  466153, 2653855, 2528002, 1400387,     124, 1031537,
       2527970, 2662066, 1930274,  980254, 2795974, 2528005, 2527979,
       2770505,  980252, 2527984, 2528021, 2528017, 2527990, 1930277,
       2527966, 2528016, 2528004,  980251, 2527985, 2681879, 2528026,
       1576369, 2605694,  431057,  591154,  107709, 1796921,  583355,
       1262865, 2174842, 1148786, 2597822, 1548208,  424488,  415222,
       2773457,  415221, 1804625, 1704307,  191863, 1970334,  690879,
        239935,  407226,  361055, 1658618, 1041768, 1041766,   83558,
         83560, 1457141, 1457153, 1353976,  340071,  483423,  225148,
       1651774,   83561, 2705072,   71667, 2681470,  469812, 2681469,
       1444712, 2494525,   83556, 1405396,   85991,     813,  172901,
        256847, 1932

In [6]:
lineages = pd.DataFrame()
for taxid in PVCtree_sampled_genomes['TaxID'].unique():
    if pd.isna(taxid):
        continue
    lineages = lineages.append({tax_rank: tmp_taxid 
                                 for tmp_taxid, tax_rank in ncbi.get_rank(ncbi.get_lineage(taxid)).items()},
                                ignore_index=True)
lineages = lineages.reindex(columns=['class', 'family',  'genus', 'phylum',
                                     'order', 'species', 'superkingdom']).copy()
lineages = lineages.query('superkingdom == 2').copy()

#lineages.loc[lineages['phylum']==1224, 'phylum'] = lineages.loc[lineages['phylum']==1224, 'class']

In [7]:
working_groups  = pd.read_parquet('PVC_working_eggNOG_groups.parquet')
working_trees   = pd.read_parquet('PVC_working_eggNOG_trees.parquet' )
eggNOG_taxonomy = pd.read_parquet('PVC_eggNOG_taxonomy.parquet'      )

In [8]:
def get_pairwise_distances(group_id):
    
    tree = ete3.Tree(working_trees.loc[group_id, 'tree'])

    leaf_names = []
    for count, node in enumerate(tree.traverse()):
        if node.is_leaf():
            leaf_names.append(node.name)
        else:
            node.name = 'node_%i' % count
    leaf_names = np.array(leaf_names)

    nodes         = []
    children      = []
    branch_length = []
    for node in tree.traverse():
        if not node.is_leaf():
            for child in node.get_children():
                nodes.append(         node.name)
                children.append(     child.name)
                branch_length.append(child.dist)

    branch_length_df                  = pd.DataFrame()
    branch_length_df['node']          = nodes
    branch_length_df['child']         = children
    branch_length_df['branch_length'] = branch_length

    dag  = ig.Graph.TupleList(edges=branch_length_df[['node', 
                                                      'child', 
                                                      'branch_length']].itertuples(index=False), 
                                directed=False, 
                                weights=True)
    
    dist_matrix = pd.DataFrame(index  =leaf_names, 
                               columns=leaf_names, 
                               data   =np.array(dag.shortest_paths(source=leaf_names, 
                                                                   target=leaf_names, 
                                                                   weights='weight'))
                              )
    return(dist_matrix)

In [9]:
def create_taxa_graph(dist_matrix, phyla):
    triu_indices       = np.triu_indices_from(dist_matrix, k=1)
    
    edge_list                 = pd.DataFrame()
    edge_list['phylum1']      = phyla[triu_indices[0]]
    edge_list['phylum2']      = phyla[triu_indices[1]]
    edge_list['sequence1']    = dist_matrix.index[triu_indices[0]]
    edge_list['sequence2']    = dist_matrix.index[triu_indices[1]]
    edge_list['distance']     = dist_matrix.values[triu_indices]
    edge_list['inverse_dist'] = np.e**np.negative(edge_list.distance)

    graph  = ig.Graph.TupleList(edges=edge_list[['sequence1', 
                                                 'sequence2', 
                                                 'inverse_dist']].itertuples(index=False), 
                                directed=False, 
                                weights =True)
    
    return(edge_list, graph)

In [10]:
def cles(lessers, greaters):
    #
    # https://github.com/ajschumacher/cles/blob/master/cles.py
    #
    """Common-Language Effect Size
    Probability that a random draw from `greater` is in fact greater
    than a random draw from `lesser`.
    Args:
      lesser, greater: Iterables of comparables.
    """
    if len(lessers) == 0 and len(greaters) == 0:
        raise ValueError('At least one argument must be non-empty')
    # These values are a bit arbitrary, but make some sense.
    # (It might be appropriate to warn for these cases.)
    if len(lessers) == 0:
        return 1
    if len(greaters) == 0:
        return 0
    numerator = 0
    lessers, greaters = sorted(lessers), sorted(greaters)
    lesser_index = 0
    for greater in greaters:
        while lesser_index < len(lessers) and lessers[lesser_index] < greater:
            lesser_index += 1
        numerator += lesser_index  # the count less than the greater
    denominator = len(lessers) * len(greaters)
    return float(numerator) / denominator

In [11]:
def assess_cluster(reference_phylum, minimal_freq_phyla, cluster_edges, cluster_nodes):

    #
    # store distances between reference phylum and others
    cluster_dists = pd.DataFrame(columns=['phylum', 'median', 'distances'])

    #
    # traverse all phylum pairs containing the reference phylum
    for phylum in minimal_freq_phyla:
        if phylum == reference_phylum:
            continue

        inter_phyla = cluster_edges.loc[((cluster_edges.phylum1==reference_phylum) & (cluster_edges.phylum2==phylum)) |\
                                        ((cluster_edges.phylum2==reference_phylum) & (cluster_edges.phylum1==phylum))]
        
        #
        # create a quadratic matrix of pairwise distances between phyla
        #   rows and columns must be unique pairs of sequence names
        indices     = np.unique(inter_phyla[['sequence1', 'sequence2']])
        adjacencies = pd.DataFrame(index  =indices, 
                                   columns=indices,
                                   data   =0.0)
        
        #
        # add distances between sequences from the edge list to the quadratic matrix
        #   as the matrix is quadratic, add values to both directions
        indexer     = adjacencies.index.get_indexer
        adjacencies.values[indexer(inter_phyla.sequence1), indexer(inter_phyla.sequence2)] = inter_phyla.distance.values
        adjacencies.values[indexer(inter_phyla.sequence2), indexer(inter_phyla.sequence1)] = inter_phyla.distance.values

        #
        # sum the obtained distances into a single cell
        tmp_closest_to_phylum = adjacencies.loc[cluster_nodes.loc[cluster_nodes.phylum==reference_phylum, 'name'],
                                                cluster_nodes.loc[cluster_nodes.phylum==phylum,           'name']].sum()
        tmp_closest_to_phylum.sort_values(inplace=True)
        #
        # and grab the five sequences from <phylum> closest to all from <reference phylum>
        tmp_closest_to_phylum = tmp_closest_to_phylum.index[:5]

        try:
            #
            # get all inter-phyla distances between
            distances_to_reference_phylum = adjacencies.loc[
                #
                # all sequences from <reference phylum>
                cluster_nodes.loc[cluster_nodes.phylum==reference_phylum, 'name'],
                #
                # the 5 sequences from <phylum> closest to <reference phylum>
                tmp_closest_to_phylum
            ].values.flatten()
        except IndexError:
            continue        

        cluster_dists = cluster_dists.append(pd.Series(data =[phylum, 
                                                              np.median(distances_to_reference_phylum), 
                                                              distances_to_reference_phylum], 
                                                       index=['phylum', 'median', 'distances']),
                                             ignore_index=True)
    return(cluster_dists)

In [12]:
def get_phyla_evol_distances(group_id):    
    dist_matrix = get_pairwise_distances(group_id)

    taxids = [int(leaf.split('.')[0]) for leaf in dist_matrix.index]
    phyla  = eggNOG_taxonomy.loc[taxids, 'phylum'].values.astype(int)

    edge_list, graph  = create_taxa_graph(dist_matrix, phyla)

    random.seed(12345)
    clusters = graph.community_multilevel(weights='weight')

    node_data = pd.DataFrame(columns=['name', 'phylum', 'cluster'],
                             data   =zip(dist_matrix.index, 
                                         phyla, 
                                         clusters.membership)
                            )
    
    cluster_evol_relations = {}
    
    target_phyla = {74201,    # Verrucomicrobia
                    256845,    # Lentisphaerae
                    204428,    # Chlamydiae
                    265317,  # Candidatus Poribacteria
                    203682} # Planctomycetes
    
    #target_phyla = {1090,    # chlorobi
    #                1117,    # cyanobacteria
    #                200795,  # chloroflexi
    #                976,     # bacteroidetes #
    #                1134404, # ignavibacteriae #
    #                1798710, # melainabacteria #
    #                28211,   # Alphaproteobacteria
    #                28216,   # Betaproteobacteria
    #                1236,    # Gammaproteobacteria
    #                28221,   # Deltaproteobacteria
    #                29547,   # Epsilonproteobacteria #
    #                580370,  # Zetaproteobacteria #
    #                1807140} # Acidithiobacillia #
    
    #target_phyla = { 28211,          # Alpha
    #                 28216,          # Beta
    #                 28221,          # Delta
    #                 1236            # Gamma
    #               }
    
    #target_phyla = {1090,    # chlorobi
    #                1117,    # cyanobacteria
    #                1224,    # proteobacteria
    #                200795,  # chloroflexi
    #                976,     # bacteroidetes
    #                1134404, # ignavibacteriae
    #                1798710} # melainabacteria

    for cluster_num in set(clusters.membership):
        
        cluster_nodes      = node_data[node_data.cluster==cluster_num]
        minimal_freq_phyla = [phylum 
                              for phylum, frequency in Counter(cluster_nodes.phylum).items() 
                              if frequency>=5   # at least five sequences from a phylum
                              and phylum > 0]   # given pandas manipulation, unknown phyla are represented 
                                                #   as NAN, which when forced to be int are negative numbers
                                                #   that is why we ignore phylum taxids smaller than zero...
       
        #
        # if there are fewer than two phyla of interested within the tree cluster there is no reason to
        #   assess it...
        if len( target_phyla.intersection( minimal_freq_phyla ) ) < 2:
            continue
        
        cluster_evol_relations[cluster_num] = {}
        
        #
        # filter patristic distances from the whole tree to only those between sequences within the cluster
        cluster_edges = edge_list.loc[(edge_list.sequence1.isin(cluster_nodes.name)) &
                                      (edge_list.sequence2.isin(cluster_nodes.name)),
                                      ['phylum1', 'phylum2', 'sequence1', 'sequence2', 'distance']]

        #
        # filter again, leaving only between sequences whose phylum fulfills the minimal frequency
        cluster_edges      = cluster_edges[(cluster_edges.phylum1.isin(minimal_freq_phyla)) &\
                                           (cluster_edges.phylum2.isin(minimal_freq_phyla))]
        #
        # we will divide all pairwise distances by the cluster's mean
        normalizer         = np.median(cluster_edges.distance)
        #
        # remove all intra-phylum distances
        cluster_edges      = cluster_edges[cluster_edges.phylum1 != cluster_edges.phylum2] 

        #
        # assess all inter-phyla distances, using each phylum as a reference to itself
        #
        for ref_phylum in target_phyla.intersection(minimal_freq_phyla):
            cluster_dists = assess_cluster(ref_phylum, 
                                           minimal_freq_phyla, 
                                           cluster_edges,
                                           cluster_nodes)

            #
            # sort phyla by its avg. distance to <reference phylum>
            cluster_dists.sort_values('median', inplace=True)
            cluster_evol_relations[cluster_num][ref_phylum] = {
                'df'         :cluster_dists[['phylum', 'median']].copy(),
                'significant':False
            }
            if not cluster_dists.shape[0]:
                continue

            cluster_evol_relations[cluster_num][ref_phylum]['df']['median']   /= normalizer
            
            #
            # if there is only one <phylum> together with <reference phylum>, the proximity between
            #   both is automaticaly significant
            if cluster_dists.shape[0] == 1:
                cluster_evol_relations[cluster_num][ref_phylum]['significant'] = True
                continue

            try:
                #
                # test if distances from the closest phylum to <reference phylum> is significantly
                #   smaller than distances from the second closest phylum
                hypothesis = mannwhitneyu(cluster_dists.iloc[0, 2], 
                                          cluster_dists.iloc[1, 2], 
                                          alternative='less')
            except ValueError:
                continue
            else:
                #
                # both cles lines below should work identically... I am leaving the above because it is the one I
                #   used in the paper, no real reason...
                effect_size = hypothesis.statistic / (len(cluster_dists.iloc[0, 2])*len(cluster_dists.iloc[1, 2]))
#                 effect_size = 1-cles(cluster_dists.iloc[0, 2], cluster_dists.iloc[1, 2])

                if hypothesis.pvalue < 0.01 and effect_size < 0.2:
                    cluster_evol_relations[cluster_num][ref_phylum]['significant'] = True
    
    return(group_id, cluster_evol_relations)

In [13]:
print(len(working_groups.group_id.values))

2985


In [14]:
%%time
pool    = multiprocessing.Pool(processes=10, maxtasksperchild=5)
results = pool.map_async(get_phyla_evol_distances, working_groups.group_id.values)
pool.close()
pool.join()

CPU times: user 11.3 s, sys: 5.55 s, total: 16.9 s
Wall time: 2h 14min 26s


In [15]:
with open('PVC_new_phyla_results.pkl', 'wb') as out:
    pkl.dump(results.get(), out)
del(results)